In [240]:
import numpy as np
import pandas as pd
import datetime
import plotly.graph_objects as go
import dash
from dash import dcc
from dash import html
import dash_table
from dash.dependencies import Input,Output,State
import warnings
warnings.filterwarnings("ignore")

# Importing Cleaned Data

In [241]:
# Importing Sales Data
aw_sales = pd.read_csv('AW Sales.csv')
aw_sales['OrderDate'] = pd.to_datetime(aw_sales['OrderDate'])
aw_sales.drop('StockDate',axis=1,inplace=True)
aw_sales.head()

,OrderDate,OrderNumber,ProductKey,CustomerKey,OrderLineItem,OrderQuantity,Region,Country,Continent
0,2020-01-01,SO45080,332,14657,1,1,Northwest,United States,North America
1,2020-01-04,SO45098,310,29167,1,1,Northwest,United States,North America
2,2020-01-04,SO45099,312,29174,1,1,Northwest,United States,North America
3,2020-01-06,SO45103,310,29140,1,1,Northwest,United States,North America
4,2020-01-08,SO45117,342,14727,1,1,Northwest,United States,North America


In [242]:
# Importing Customers Data
aw_customers = pd.read_csv("AW Customers.csv")
aw_customers['BirthDate'] = pd.to_datetime(aw_customers['BirthDate'])
aw_customers.head()

,CustomerKey,FirstName,LastName,BirthDate,CustomerAge,MaritalStatus,Gender,EmailAddress,AnnualIncome,TotalChildren,EducationLevel,Occupation,HomeOwner,FullName,AgeGroup,IncomeLevel,EducationCategory,CustomerPriority
0,11000,Jon,Yang,1966-04-08,57,M,M,jon24@adventure-works.com,90000,2,Bachelors,Professional,Y,Jon Yang,50-59,Average,Bachelors,Standard
1,11001,Eugene,Huang,1965-05-14,58,S,M,eugene10@adventure-works.com,60000,3,Bachelors,Professional,N,Eugene Huang,50-59,Average,Bachelors,Standard
2,11002,Ruben,Torres,1965-08-12,58,M,M,ruben35@adventure-works.com,60000,3,Bachelors,Professional,Y,Ruben Torres,50-59,Average,Bachelors,Standard
3,11003,Christy,Zhu,1968-02-15,55,S,F,christy12@adventure-works.com,70000,0,Bachelors,Professional,N,Christy Zhu,50-59,Average,Bachelors,Standard
4,11004,Elizabeth,Johnson,1968-08-08,55,S,F,elizabeth5@adventure-works.com,80000,5,Bachelors,Professional,Y,Elizabeth Johnson,50-59,Average,Bachelors,Standard


In [243]:
# Importing Products Data
aw_products = pd.read_csv("AW Products.csv")
aw_products['ProductStyle'] = aw_products['ProductStyle'].fillna('Unknown')
aw_products.head()

,ProductKey,ProductSKU,SKU Type,ProductName,ModelName,ProductDescription,ProductColor,ProductStyle,ProductCost,ProductPrice,SubcategoryName,CategoryName
0,214,HL-U509-R,HL-U509,"Sport-100 Helmet, Red",Sport-100,"Universal fit, well-vented, lightweight , snap...",Red,Unknown,13.09,34.99,Helmets,Accessories
1,215,HL-U509,HL-U509,"Sport-100 Helmet, Black",Sport-100,"Universal fit, well-vented, lightweight , snap...",Black,Unknown,12.03,33.64,Helmets,Accessories
2,220,HL-U509-B,HL-U509,"Sport-100 Helmet, Blue",Sport-100,"Universal fit, well-vented, lightweight , snap...",Blue,Unknown,12.03,33.64,Helmets,Accessories
3,446,PA-T100,PA-T100,"Touring-Panniers, Large",Touring-Panniers,"Durable, water-proof nylon construction with e...",Grey,Unknown,51.56,125.00,Panniers,Accessories
4,447,LO-C100,LO-C100,Cable Lock,Cable Lock,"Wraps to fit front and rear tires, carrier and...",unknown,Unknown,10.31,25.00,Locks,Accessories


In [244]:
# Importing Returns Data
aw_returns = pd.read_csv("AW Returns.csv")
aw_returns['ReturnDate'] = pd.to_datetime(aw_returns['ReturnDate'])
aw_returns.head()

,ReturnDate,TerritoryKey,ProductKey,ReturnQuantity
0,2020-01-18,9,312,1
1,2020-01-18,10,310,1
2,2020-01-21,8,346,1
3,2020-01-22,4,311,1
4,2020-02-02,6,312,1


# Variables for Dashboards Plots

In [245]:
# For Revenue Trend
df1 = aw_sales.merge(aw_products,on='ProductKey',how='inner')[['OrderDate','OrderQuantity','ProductPrice']]
df1['Revenue'] = df1['OrderQuantity'] * df1['ProductPrice']
df1.set_index('OrderDate',inplace=True)
df1['Year'] = df1.index.year
df1.head()

,OrderQuantity,ProductPrice,Revenue,Year
OrderDate,,,,
2020-01-01,1,699.1,699.1,2020
2020-02-06,1,699.1,699.1,2020
2020-07-27,1,699.1,699.1,2020
2020-08-27,1,699.1,699.1,2020
2020-09-09,1,699.1,699.1,2020


In [246]:
# For Orders by Category Plot
df2 = aw_sales.merge(aw_products,on='ProductKey',how='inner')[['OrderDate','CategoryName']].set_index('OrderDate')
df3 = df2.groupby('CategoryName').resample('Y').count()

In [247]:
df3['Year'] = df3.index.get_level_values(1).year
df3.query('Year==2022').index.get_level_values(0)

Index(['Accessories', 'Bikes', 'Clothing'], dtype='object', name='CategoryName')

In [248]:
df3.rename(columns={'CategoryName':'OrdersCount'},inplace=True)

In [249]:
df3.reset_index(inplace=True)

In [250]:
# Total Revenue for all all years
total_revenue = df1['Revenue'].sum()

In [251]:
# Calculation for Total Profit
df4 = aw_sales.merge(aw_products,on='ProductKey',how='inner')[['OrderQuantity','ProductCost']]
total_cost = (df4['OrderQuantity']*df4['ProductCost']).sum()
profit = total_revenue - total_cost
profit

10457580.860000001

In [252]:
# Calculation for Total Orders
orders = aw_sales['OrderNumber'].nunique()
orders

25164

In [253]:
# Calculation for Return Rate
return_rate = aw_returns['ReturnQuantity'].sum()/aw_sales['OrderQuantity'].sum()
return_rate

0.021716919713925916

In [254]:
# Calculation for top 10 products by orders
df5 = aw_sales.merge(aw_products,on='ProductKey')[['OrderDate','OrderNumber','ProductName','OrderQuantity','ProductCost','ProductPrice']]
df5

,OrderDate,OrderNumber,ProductName,OrderQuantity,ProductCost,ProductPrice
0,2020-01-01,SO45080,"Road-650 Black, 58",1,413.15,699.10
1,2020-02-06,SO45383,"Road-650 Black, 58",1,413.15,699.10
2,2020-07-27,SO46896,"Road-650 Black, 58",1,413.15,699.10
3,2020-08-27,SO47311,"Road-650 Black, 58",1,413.15,699.10
4,2020-09-09,SO47507,"Road-650 Black, 58",1,413.15,699.10
...,...,...,...,...,...,...
56041,2022-04-28,SO69240,"Touring-1000 Yellow, 60",1,1481.94,2384.07
56042,2022-05-07,SO70086,"Touring-1000 Yellow, 60",1,1481.94,2384.07
56043,2022-05-17,SO70800,"Touring-1000 Yellow, 60",1,1481.94,2384.07
56044,2022-05-31,SO71749,"Touring-1000 Yellow, 60",1,1481.94,2384.07


In [255]:
df5['Revenue'] = df5['OrderQuantity']*df5['ProductPrice']
df5.drop(['OrderQuantity','ProductCost','ProductPrice'],axis=1,inplace=True)
df5

,OrderDate,OrderNumber,ProductName,Revenue
0,2020-01-01,SO45080,"Road-650 Black, 58",699.10
1,2020-02-06,SO45383,"Road-650 Black, 58",699.10
2,2020-07-27,SO46896,"Road-650 Black, 58",699.10
3,2020-08-27,SO47311,"Road-650 Black, 58",699.10
4,2020-09-09,SO47507,"Road-650 Black, 58",699.10
...,...,...,...,...
56041,2022-04-28,SO69240,"Touring-1000 Yellow, 60",2384.07
56042,2022-05-07,SO70086,"Touring-1000 Yellow, 60",2384.07
56043,2022-05-17,SO70800,"Touring-1000 Yellow, 60",2384.07
56044,2022-05-31,SO71749,"Touring-1000 Yellow, 60",2384.07


In [256]:
df5.groupby('ProductName').agg({
    'Revenue': 'sum',
    'OrderNumber': 'nunique'
}).reset_index()


,ProductName,Revenue,OrderNumber
0,AWC Logo Cap,35864.64,2062
1,All-Purpose Bike Stand,37206.00,234
2,Bike Wash - Dissolver,13562.70,850
3,"Classic Vest, L",11557.00,182
4,"Classic Vest, M",11557.00,182
...,...,...,...
125,"Touring-3000 Yellow, 62",35632.80,48
126,Water Bottle - 30 oz.,39755.33,3983
127,"Women's Mountain Shorts, L",23376.66,334
128,"Women's Mountain Shorts, M",22886.73,327


In [257]:
# Set Order Date as Index
df5.set_index('OrderDate', inplace=True)

# Group by Product Name Resample by Year
df_grouped = df5.groupby('ProductName').resample('Y').agg({
    'Revenue': 'sum',
    'OrderNumber': 'count'
})

# Reset the index
df_grouped.reset_index(inplace=True)

# To convert 'OrderDate' back to just the year as an integer
df_grouped['OrderDate'] = df_grouped['OrderDate'].dt.year


In [258]:
df_grouped.rename(columns={'ProductName':'Product','OrderNumber':'Orders'},inplace=True)
df_grouped.query('OrderDate==2020').sort_values(by='Orders',ascending=False).head(10)

,Product,OrderDate,Revenue,Orders
113,"Road-150 Red, 48",2020,640510.33,179
116,"Road-150 Red, 62",2020,604727.63,169
114,"Road-150 Red, 52",2020,601149.36,168
115,"Road-150 Red, 56",2020,561788.39,157
112,"Road-150 Red, 44",2020,497379.53,139
123,"Road-250 Black, 52",2020,235608.48,108
135,"Road-250 Red, 58",2020,231245.36,106
131,"Road-250 Red, 48",2020,219901.50,90
120,"Road-250 Black, 48",2020,181069.48,83
133,"Road-250 Red, 52",2020,195468.00,80


In [259]:
df_grouped['Revenue'] = df_grouped['Revenue'].round(1)
df_grouped['Revenue'] = df_grouped['Revenue'].apply(lambda x: '$'+str(x))

In [260]:
# Most Ordered Product Type 
product1 = aw_sales.merge(aw_products, on="ProductKey", how="inner")[
    ["OrderNumber", "SubcategoryName"]
].groupby("SubcategoryName").count().idxmax()

product1

OrderNumber    Tires and Tubes
dtype: object

In [261]:
# Most Returned Product Type
return_quantity = aw_returns.merge(aw_products, on="ProductKey", how="inner")[
    ["ReturnQuantity", "SubcategoryName"]
].groupby("SubcategoryName").sum()

total_orders = aw_sales.merge(aw_products, on="ProductKey", how="inner")[
    ["OrderNumber", "SubcategoryName"]
].groupby("SubcategoryName").count()

product2 = (return_quantity['ReturnQuantity']/total_orders['OrderNumber']).idxmax()
product2

'Shorts'

# Dashboard Development

In [293]:
# Creating Dash Instance
app = dash.Dash()

# App Layout
app.layout = html.Div(
    [dcc.Location(id="url", refresh=False), html.Div(id="page-content")]
)

# Index Page Layout (List Links to all Dashboards)
index_page = html.Div(
    [
        html.H1(
            "AdventureWorks Sales Dashboard",
            style={"fontSize": "40px", "color": "blue"},
        ),
        dcc.Link(
            "Executive Dashboard",
            href="/page-1",
            style={"fontSize": "25px", "color": "red"},
        ),
        html.Br(),
        dcc.Link(
            "Product Details",
            href="/page-2",
            style={"fontSize": "25px", "color": "red"},
        ),
        html.Br(),
        dcc.Link(
            "Customer Details",
            href="/page-3",
            style={"fontSize": "25px", "color": "red"},
        ),
    ]
)

# Layout for Excecutive Dashboard
page_1_layout = html.Div(
    [
        html.H1(
            "Executive Dashboard",
            style={"textAlign": "center", "fontSize": "40px", "color": "green",'text-decoration':'underline'},
        ),
        html.Div([
                html.H2(f'${round(total_revenue/1000000,1)}M',style={'fontSize':'35px'}),
                html.H4('Revenue',style={'color':'blue','fontSize':'20px'})],
                style={'border': 'thin lightgrey solid','border-radius': '5px','background-color': 'orange','padding': '30px',
                       'width': '200px','height': '100px','text-align': 'center','margin-left':'250px','margin-bottom':'40px','display':'inline-block'}),
        html.Div([
                html.H2(f'${round(profit/1000000,2)}M',style={'fontSize':'35px'}),
                html.H4('Profit',style={'color':'blue','fontSize':'20px'})],
                style={'border': 'thin lightgrey solid','border-radius': '5px','background-color': 'orange','padding': '30px',
                       'width': '200px','height': '100px','text-align': 'center','margin-left':'100px','margin-bottom':'40px','display':'inline-block'}),
        html.Div([
                html.H2(f'{round(orders/1000,1)}K',style={'fontSize':'35px'}),
                html.H4('Orders',style={'color':'blue','fontSize':'20px'})],
                style={'border': 'thin lightgrey solid','border-radius': '5px','background-color': 'orange','padding': '30px',
                       'width': '200px','height': '100px','text-align': 'center','margin-left':'100px','margin-bottom':'40px','display':'inline-block'}),
        html.Div([
                html.H2(f'{round(return_rate*100,1)}%',style={'fontSize':'35px'}),
                html.H4('Return Rate',style={'color':'blue','fontSize':'20px'})],
                style={'border': 'thin lightgrey solid','border-radius': '5px','background-color': 'orange','padding': '30px',
                       'width': '200px','height': '100px','text-align': 'center','margin-left':'100px','margin-bottom':'40px','display':'inline-block'}),
        html.Div(
            [
                html.Label(
                    "Select Year(s)",
                    style={"fontSize": "20px",'margin-left':'230px'},
                )]),
        html.Div(
            [
                dcc.RangeSlider(
                    id="slider-1",
                    min=2020,
                    max=2022,
                    step=1,
                    value=[2020, 2022],
                    marks={i: str(i) for i in range(2020, 2023)})],
                    style={"width": "30%"}),
                dcc.Graph(
                    id="revenue-trend",
                    style={"width": "48%", "display": "inline-block"}),
                dcc.Graph(
                    id="order-by-category",
                    style={"width": "50%", "display": "inline-block"}),
                html.H3('Top 10 Products by Orders and Year',style={'color':'red','margin-left':'150px','display':'inline-block'}),
                dash_table.DataTable(
                                     id='top-10-products',
                                     columns=[{'name':'Product','id':'Product'},{'name':'Orders','id':'Orders'},{'name':'Revenue','id':'Revenue'}],
                                     style_table={'height':'300px','OverflowY':'auto','width':'30%', 'display':'inline-block'},
                                     style_header={'textAlign':'center','backgroundColor':'yellow','fontWeight':'bold','fontSize':'16px'},
                                     style_data_conditional=[{'if': {'column_id': 'Product'},'textAlign': 'left'}],
                                     style_cell={'textAlign': 'center'}),
                dcc.Link("Go back to home", href="/")])

# Layout for Product Details
page_2_layout = html.Div(
    [
        html.H1(
            "Product Details",
            style={"textAlign": "center", "fontSize": "40px", "color": "blue"},
        ),
        dcc.Link("Go back to home", href="/"),
    ]
)

# Layout for Customer Details
page_3_layout = html.Div(
    [
        html.H1(
            "Customer Details",
            style={"textAlign": "center", "fontSize": "40px", "color": "blue"},
        ),
        dcc.Link("Go back to home", href="/"),
    ]
)


# Call Back Function for Index Page
@app.callback(Output("page-content", "children"), [Input("url", "pathname")])
def display_page(pathname):
    if pathname == "/page-1":
        return page_1_layout
    elif pathname == "/page-2":
        return page_2_layout
    elif pathname == "/page-3":
        return page_3_layout
    else:
        return index_page


# Callback Function for Revenue Trend (Executive Dashboard)
@app.callback(
    Output(component_id="revenue-trend", component_property="figure"),
    [Input(component_id="slider-1", component_property="value")],
)
def revenue_trend_graph(year):
    start_year = year[0]
    end_year = year[1]
    df1a = df1.query("Year>= @start_year & Year<=@end_year").resample("M").sum()
    plot1 = go.Scatter(x=df1a.index, y=df1a["Revenue"], mode="lines")
    layout1 = go.Layout(
        title={"text": "Revenue Trend", "font": {"color": "red"}},
        xaxis={
            "title": "Order Date",
            "titlefont": {"color": "red"},
            "tickfont": {"color": "blue"},
        },
        yaxis={
            "title": "Revenue",
            "titlefont": {"color": "red"},
            "tickfont": {"color": "blue"},
        },
    )
    return {"data": [plot1], "layout": layout1}


# Callback Function for Orders by Category Trend (Executive Dashboard)
@app.callback(
    Output(component_id="order-by-category", component_property="figure"),
    [Input(component_id="slider-1", component_property="value")],
)
def graph_order_by_category(year):
    start_year1 = year[0]
    end_year1 = year[1]
    df4 = (
        df3.query("Year>=@start_year1 & Year<=@end_year1")[
            ["CategoryName", "OrdersCount"]
        ]
        .groupby("CategoryName")
        .sum()
    )
    plot2 = go.Bar(x=df4["OrdersCount"], y=df4.index, orientation="h")
    layout2 = go.Layout(
        title={"text": "Total Orders By Category and Year", "font": {"color": "red"}},
        xaxis={
            "title": "Total Orders",
            "titlefont": {"color": "red"},
            "tickfont": {"color": "blue"},
        },
        yaxis={
            "title": "Category Name",
            "titlefont": {"color": "red"},
            "tickfont": {"color": "blue"},
        },
    )
    return {"data": [plot2], "layout": layout2}

# Callback Function for Top 10 Products Table
@app.callback(Output(component_id='top-10-products',component_property='data'),
              [Input(component_id='slider-1',component_property='value')])

def top_10_products_table(year):
    start_year3 = year[0]
    end_year3 = year[1]
    df6 = df_grouped.query('OrderDate>=@start_year3 & OrderDate<=@end_year3').sort_values(by='Orders',ascending=False).head(10)
    return df6.to_dict('records')


# Run your app
if __name__ == "__main__":
    app.run_server()


Dash is running on http://127.0.0.1:8050/



INFO: Dash is running on http://127.0.0.1:8050/



 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
INFO: WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
INFO: Press CTRL+C to quit
127.0.0.1 - - [09/Jul/2023 13:20:30] "GET /page-1 HTTP/1.1" 200 -
INFO: 127.0.0.1 - - [09/Jul/2023 13:20:30] "GET /page-1 HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2023 13:20:30] "GET /_dash-layout HTTP/1.1" 200 -
INFO: 127.0.0.1 - - [09/Jul/2023 13:20:30] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2023 13:20:30] "GET /_dash-dependencies HTTP/1.1" 200 -
INFO: 127.0.0.1 - - [09/Jul/2023 13:20:30] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2023 13:20:30] "POST /_dash-update-component HTTP/1.1" 200 -
INFO: 127.0.0.1 - - [09/Jul/2023 13:20:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2023 13:20:30] "GET /_favicon.ico?v=2.9.2 HTTP/1.1" 200 -
INFO: 127.0.0.1 - - [09/Jul/2023 13:20:30] "GET /_fav